# Try different models-compare clustering results over eval. set
TODO: try different eps

In [46]:
# load eval.set (duplicates free)
import json
filename = 'english_3_clusters.json'
#filename = 'set_with_noise.json'
with open(filename, 'r') as file:
    cdocs = json.load(file)
print("#Number of clusters:",len(cdocs))
print("##Cluster length:",[str(len(cluster)) for cluster in cdocs])
print("### Doc length: ",[len(doc['headline']+doc['bodyText']) for cluster in cdocs for doc in cluster 
                          if not(doc['bodyText'] is None)])
print(sum([len(doc['headline']+doc['bodyText']) for cluster in cdocs for doc in cluster])/60)

#Number of clusters: 3
##Cluster length: ['20', '20', '20']
### Doc length:  [10892, 5326, 4735, 3605, 7054, 1028, 1205, 10558, 4099, 2532, 5444, 3866, 1187, 3702, 5434, 261, 10860, 1859, 1325, 5784, 6009, 2788, 3228, 1736, 3098, 3075, 3638, 3608, 3619, 6182, 4331, 4516, 2265, 6123, 3943, 5139, 2874, 3922, 5734, 4247, 2320, 3093, 7587, 2811, 4230, 2806, 2622, 5009, 5855, 2910, 4416, 5706, 3281, 3273, 3973, 4828, 3675, 7436, 6775, 4819]
4304.266666666666


In [12]:
#print([d for (d, i) in train_corpus_ta][:10])

## Title+abstract models
## TODO: lower case text

In [39]:
# train different models
from gensim.utils import simple_preprocess as sp
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing

unwrapped_docs = [doc for cluster in cdocs for doc in cluster]

# remove noise from expected clusters 
noise = [cluster for cluster in cdocs if len(cluster)==1]
cdocs = [cluster for cluster in cdocs if not(cluster in noise)]
print("Noise length {} and cdocs length {}".format(len(noise), len(cdocs)))

import random
random.shuffle(unwrapped_docs)
# title + abstract models; here abstract is called 'content' (I got these articles from GuardianAPI)
train_corpus_ta = [ TaggedDocument(sp(doc['headline'] + doc['bodyText']) ,[i]) for i, doc in enumerate(unwrapped_docs)]
print("Total number of documents:",len(train_corpus_ta))
#print(train_corpus_ta)
print([len(doc.words) for doc in train_corpus_ta])
epochs = 45
vec_size = 100
models = [
    # dm = 0, simple SG, simpler model, most of the time efficient and accurate
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 100'),
    Doc2Vec(dm=0, vector_size=800, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800, +Eps'),
    Doc2Vec(dm=0, vector_size=800, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800'),
    Doc2Vec(dm=0, vector_size=1000, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 1000'),
  
    # PV-DM
    Doc2Vec(dm=1, vector_size= 500, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=500, alpha=.05'),
]

# build our vocabulary of words (all the unique words encountered inside our corpus)
for model in models:
    model.build_vocab(train_corpus_ta)
    print(model, "-VocabSize:", len(model.wv.vocab))
print("Vocabulary created!")

# train the models on the given data!
counter = 0
for model in models:
    print("Training %s" % model)
    %time model.train(train_corpus_ta, total_examples=len(train_corpus_ta), epochs=model.epochs)
    #model.save(MODEL_NAME+str(counter)+'.model')
    counter = counter + 1
#print("Models Saved")

Noise length 0 and cdocs length 3
Total number of documents: 60
[479, 1150, 165, 1240, 481, 196, 595, 895, 524, 642, 424, 668, 357, 788, 251, 1759, 611, 289, 1198, 710, 623, 658, 490, 1058, 45, 511, 534, 570, 421, 1093, 904, 864, 695, 904, 609, 201, 441, 640, 770, 414, 798, 769, 656, 623, 200, 750, 1805, 844, 948, 1738, 347, 521, 604, 907, 471, 961, 907, 574, 710, 441]
Doc2Vec("dm=0, vec = 100",dbow,d100,n5,mc2,t4) -VocabSize: 3093
Doc2Vec("dm=0, vec = 800, +Eps",dbow,d800,n5,mc2,t4) -VocabSize: 3093
Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4) -VocabSize: 3093
Doc2Vec("dm=0, vec = 1000",dbow,d1000,n5,mc2,t4) -VocabSize: 3093
Doc2Vec("dm=1, vec=500, alpha=.05",dm/m,d500,n5,w10,mc2,t4) -VocabSize: 3093
Vocabulary created!
Training Doc2Vec("dm=0, vec = 100",dbow,d100,n5,mc2,t4)
CPU times: user 4.27 s, sys: 24 ms, total: 4.29 s
Wall time: 1.53 s
Training Doc2Vec("dm=0, vec = 800, +Eps",dbow,d800,n5,mc2,t4)
CPU times: user 13.6 s, sys: 64 ms, total: 13.7 s
Wall time: 4.29 s
Training Doc2

# Evaluate Clustering

In [41]:
import utils
import my_dbscan
import model_evaluation as me
#import importlib
#mportlib.reload(me)

models_recall = []
models_precision = []
clusters_found = [] # keep clusters results, they're useful later on
min_s = 2 # min_samples
titles = [doc['headline'] for doc in unwrapped_docs]
urls = [doc['headline'] for doc in unwrapped_docs] # I don't have url in the data-set
    
for k, model in enumerate(models):
    # try different eps for some model
    if k == 1:
        eps = 0.35
        eps_increment = 0.1
        min_s = 2
    else:
        eps = 0.11
        eps_increment = 0.13
        min_s = 2
    # get list of document vectors
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    
    # apply dbscan clustering to these vectors
    titles_clusters = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = eps, eps_increment = eps_increment,
                                               n_iterations = 3, verbose = False, min_samples = min_s)
    # get clusters as list of titles
    #titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
    clusters_found.append(titles_clusters)
    #data = utils.plot_clusters(titles_clusters)
    
    # evaluate clustering
    expected_clusters = []
    for docs in cdocs:
        expected_clusters.append([doc['headline'] for doc in docs])
    precision, recall = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
    print('Precision score: %s, Recall score: %s'%(precision, recall))
    
    models_precision.append(precision)
    models_recall.append(recall)
    # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
    # that's why we have precision score
    print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
        len([l for l in titles_clusters if len(l)>0]), len(cdocs)))
    
    # noise classified
    c = 0
    for n in noise:
        for cluster in titles_clusters:
            if n[0]['headline'] in cluster:
                c += 1
    print("Classified {} noise articles among the total of {}".format(c, len(noise)))


Accuracy (Precision) over each cluster:  [94.44444444444446, 83.33333333333333, 100.0]
Accuracy (Recall) over each cluster:  [15.0, 13.0, 16.666666666666668]
Precision score: 92.5925925925926, Recall score: 14.888888888888891
#Number of clusters found: 10, against number of pre-computed clusters: 3#

Classified 0 noise articles among the total of 0
Accuracy (Precision) over each cluster:  [86.80555555555556, 77.43055555555554, 90.47619047619048]
Accuracy (Recall) over each cluster:  [25.0, 12.5, 12.857142857142858]
Precision score: 84.90410052910052, Recall score: 16.78571428571429
#Number of clusters found: 16, against number of pre-computed clusters: 3#

Classified 0 noise articles among the total of 0
Accuracy (Precision) over each cluster:  [92.85714285714286, 84.52380952380952, 76.78571428571429]
Accuracy (Recall) over each cluster:  [35.0, 13.333333333333334, 16.25]
Precision score: 84.72222222222223, Recall score: 21.527777777777782
#Number of clusters found: 10, against number 

In [45]:
for cluster in clusters_found[2]:
    print(cluster)
print([len(c) for c in clusters_found])

['Don’t let bacteria-laden humans contaminate Mars', 'Could the Earth lose its atmosphere as Mars once did?']
["Koreas' Kim and Moon to meet again as Trump nuclear agenda sidelined", 'North Korea set to allow inspectors into nuclear test site']
['North Korea threat remains despite Trump summit, says Japan', 'Japan defence ministry seeks record budget over North Korea threat']
['Over-65s to be given more effective flu vaccine', 'NHS denies flu vaccine shortage amid complaints over delays']
['Congo turmoil means Ebola vaccinators will need armed escorts, experts warn', 'DRC: experimental Ebola vaccine to be administered in Mbandaka']
['North Korea is still developing nuclear weapons, says IAEA', 'Pompeo backs away from North Korea denuclearisation timeline', 'South Korea and US resume military drills ahead of nuclear talks', 'Mike Pompeo to visit North Korea this week for nuclear talks', "North Korea frustrates US as 'maximum pressure' eases on sanctions", 'US may resume war games as Nor

# Conclusions
Here, you can clearly see how important the clustering alg. is for obtaining good results: the data-set is (objectively) okay, it has more than enough words, and each model gets a pretty high Precision score, which means that (almost) every document that is put together rightfully belongs to the same cluster, as you can see in the clusters; the 'problem' is that we're splitting clusters too much, finding meaningful, but somewhat unecessary sub-clusters, very specific to a single topic, which is not a bad thing.. it means that by tweaking that 'eps' we can get pretty different results.. is there a way to choose the eps value automatically?

In [32]:
import utils
import importlib
importlib.reload(utils)
# finally, call eps-estimate function
doc_vecs = [models[1].docvecs[j] for j in range(len(models[1].docvecs))]
title_dist_tuples = utils.choose_eps(2, doc_vecs, titles)
data = utils.visualize_eps_graph(title_dist_tuples=title_dist_tuples)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.tools as tls
import plotly.graph_objs as go

tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')
py.iplot(data, filename='aa')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~D4nt3_/0 or inside your plot.ly account where it is named 'aa'


# Visualize results graphically

In [44]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3_', api_key='4O71urldgOueVtcApOdX')
graph_name = 'model_eval_result_3_cl'

model_descr = [model.comment for model in models]
# plot test-accuracy results, plus silhoutte scores (times 100, since all the values are in between 0-1)
trace0 = go.Bar(
    x = model_descr,
    y = models_precision,
    name='Precision',
    marker=dict(
        color='rgb(49,130,189)'
    )
)

trace1 = go.Bar(
    x = model_descr,
    y = models_recall,
    name='Recall',
    marker=dict(
        color='rgb(155, 244, 66)',
    )
    
)
# precision, recall, silhoutte
data = [trace0, trace1]
layout = go.Layout(
    title = 'Clustering Accuracy Results',
    xaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)',
            
        ),
        tickangle = -45
    ),
    yaxis=dict(
        title='Clustering accuracy (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
   
    barmode='group',
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename=graph_name)

# Silhoutte score
Silhouette coefficient combines ideas of both cohesion and separation, 
but for individual points, as well as clusters and 
clusterings; the Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
The Silhoutte score is merely the average of each silhoutte coefficient, computed over each sample.
It's a measure of the goodness of clustering, by assuming the fact that a cluster X is defined good if both every sample inside it is close to each other, and far from any other relatively-near cluster Y.
This is not necessarily true for every shape of cluster.

    The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [15]:
from sklearn.metrics import silhouette_score
s_scores = []
for model in models:
    # get clusters for each model
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = 0.27, eps_increment = 0.1, n_iterations = 3, verbose = False)
    
    # get cluster labels, mantaining original docs ordering
    labels = []
    for doc in unwrapped_docs:
        noise = True
        for i, url_list in enumerate(urls_cluster_list):
            if(doc['url'] in url_list):
                labels.append(i) # keep cluster id
                noise = False
        if noise:
            labels.append(-1)
    # make sure they have the same size
    assert len(labels) == len(doc_vecs)
    ss = silhouette_score(doc_vecs, labels , metric='cosine')
    s_scores.append(ss)
    print(ss)

KeyError: 'url'

# Conclusions: 
A really low silhoutte score helps us identify models that tend to have very few cluster, hence not really recognizing differences between docs. This is fundamental, since test rules used so far prevent us from recognizing these kinds of models.
Since silhoutte score is an average of silhoutte coefficients, smaller changes of values here may mean greater differences.